# Structural Imbalance with the D-Wave System

## Imports

In [1]:
import networkx as nx
import random
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave_networkx as dnx
from helpers.draw import draw
from dwave_structural_imbalance_demo.mmp_network.loader import *

Loading BokehJS ...

## Formulating the Problem for a D-Wave System

D-Wave systems solve Ising problems: given $N$ variables $s_1,...,s_N$, where each variable $s_i$ can have values $-1$ or $+1$, the system finds assignments of values that minimize 

  $\sum_{i=1}^N h_i s_i +
  \sum_{i<j}^N J_{i,j} s_i s_j$,

where $h_i$ and $J_{i,j}$ are configurable (linear and quadratic) coefficients.

In our case, variables $s_i$ can represent people, with values $-1,+1$ denoting a person's assignment to one of the two sets we want to divide the social network into. If we set $J_{i,j}$ to $-1$ for friendly $s_is_j$ pairs and $+1$ for hostile pairs, their multiplication takes values

$J_{i,j} s_i s_j=
\begin{cases} 
      -1 & \text{friends in same set (} s_i=s_j \text{) or enemies in different sets ($s_i \ne s_j$)} \\
      +1 & \text{friends in different sets ($s_i \ne s_j$) or enemies in same set ($s_i=s_j$)} 
\end{cases}
$

The summation $\sum_{i<j}^N J_{i,j} s_i s_j$ now decrements when an assignment contributes to balance and increments when it contributes to frustration.

The quantum computer finds partitions (assignments of $s_i$) that minimize frustration.

## A Real-World Example

A study of the violent extremist network in Syria found that the network was balanced in 2012. However, in 2013 an increase in active groups in the Syrian theatre changed the existing landscape significantly.

In [27]:
sampler = EmbeddingComposite(DWaveSampler(endpoint='https://cloud.dwavesys.com/sapi', \
                                          token='DEV-bc7705c39af039eacfdfe447749da106f22ab4f2', \
                                          solver='DW_2000Q_2_1'))

In [28]:
G = global_signed_social_network()

In [29]:
# Select the Syria subregion by creating subgraph S from the full data set G
syria_groups = set()
for v, data in G.nodes(data=True):
    if 'map' not in data:
        continue
    if data['map'] in {'Syria', 'Aleppo'}:
        syria_groups.add(v)
S = G.subgraph(syria_groups)

# Filter by year
year = 2013
filtered_edges = ((u, v) for u, v, a in S.edges(data=True) if a['event_year'] <= year)
S = S.edge_subgraph(filtered_edges)

In [30]:
position = draw(S)

In [31]:
# Return a good partition of the Syrian 2013 network and its frustrated edges 
imbalance, bicoloring = dnx.structural_imbalance(S, sampler)
# Annotate the network with the returned frustrated edges and node sets
for edge in S.edges:
    S.edges[edge]['frustrated'] = edge in imbalance
for node in S.nodes:
    S.nodes[node]['color'] = bicoloring[node]

Redraw the network with the previous node positioning: nodes are now bicolored and dashed lines indicate frustrated edges.

In [32]:
draw(S, position);

Redraw the network with a new positioning that separates the two sets.

In [23]:
# Frustrated edges now stand out
draw(S);

## What about a classical graph algorithm?

In [33]:
import dimod
solver = dimod.ExactSolver()

In [34]:
# Return a good partition of the Syrian 2013 network and its frustrated edges 
imbalance, bicoloring = dnx.structural_imbalance(G, solver)
# Annotate the network with the returned frustrated edges and node sets
for edge in S.edges:
    S.edges[edge]['frustrated'] = edge in imbalance
for node in S.nodes:
    S.nodes[node]['color'] = bicoloring[node]

KeyboardInterrupt: 